# PRACTICAL NO. 2

## Tackle Noise with Error Correction

In [1]:
!pip install qiskit qiskit-aer

In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import random

In [3]:
qc = QuantumCircuit(3,1)

In [4]:
# Qubit number 0
qc.x(0)
# Qubit number 1
qc.cx(0, 1)
# Qubit number 2
qc.cx(0, 2)

In [5]:
error_qubit = random.choice([0, 1, 2])
qc.x(error_qubit)
print(f"Noise introduced on qubit {error_qubit}")

Noise introduced on qubit 0


In [6]:
# Toffoli Gate
qc.cx(0,1)
qc.cx(0,2)
qc.ccx(1,2,0)

In [7]:
qc.measure(0, 0)

In [8]:
simulator = AerSimulator()
compiled = transpile(qc, simulator)
result = simulator.run(compiled, shots=1024).result()
counts = result.get_counts()

In [9]:
print("\nMeasurement results:", counts)


Measurement results: {'1': 1024}


In [10]:
# Yes, I **understand** the provided Jupyter Notebook code.

# This code implements a basic version of a **Quantum Error Correction (QEC) scheme**, specifically a simplified **Bit-Flip Code**, using Qiskit. The goal is to protect a single qubit's state from a simple noise error (a bit flip, $X$ gate).

# Here is an explanation of the code, structured to prepare you for questions you might face in your practical exam.

# -----

# ## 💻 Quantum Error Correction (QEC) Practical

# This practical demonstrates how to encode a quantum state, introduce an error (noise), 
#                             and then use a recovery circuit to correct the error, protecting the original information.

# ### 1\. Quantum Bit-Flip Code: Theory

# The code uses a minimal, three-qubit **Repetition Code** to correct a single **Bit-Flip error** (an $X$ error).

#   * **Goal:** Protect the state of the logical qubit, $|\psi_L\rangle$, from a single $X$ error.
#   * **Logical Qubit:** The original information is stored in **one** qubit, say $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$.
#   * **Encoding:** The single qubit is encoded into a **three-qubit state** (a redundant encoding).
#       * $|0\rangle \rightarrow |0_L\rangle = |000\rangle$
#       * $|1\rangle \rightarrow |1_L\rangle = |111\rangle$
#       * The original state $|\psi\rangle$ is encoded as $|\psi_L\rangle = \alpha|000\rangle + \beta|111\rangle$.

# ### 2\. Code Breakdown and Explanation

# | Code Segment | Qiskit Action | Explanation |
# | :--- | :--- | :--- |
# | `qc = QuantumCircuit(3, 1)` | Creates a circuit. | Initializes a quantum circuit with **3 quantum qubits** (data qubits) and 
# **1 classical bit** for the final measurement result. |
# | `qc.x(0)` | Applies an $X$ gate. | Sets the initial state of the first qubit (the data qubit) to $|1\rangle$. This means the 
# logical state we are encoding is $|1_L\rangle = |111\rangle$. |
# | `qc.cx(0, 1)` <br> `qc.cx(0, 2)` | Applies two CNOT gates. | This is the **Encoding Circuit**. 
# It copies the state of qubit 0 onto qubits 1 and 2, creating the logical state $|111\rangle$ from $|1\rangle$. |
# | `error_qubit = random.choice([0, 1, 2])`<br>`qc.x(error_qubit)` | Applies a random $X$ gate. 
# | This simulates **Noise**. An $X$ (bit-flip) error is applied to *one* of the three qubits randomly. For example, 
# if $error\_qubit=1$, the state changes from $|111\rangle$ to $|101\rangle$. |
# | `qc.cx(0,1)` <br> `qc.cx(0,2)` <br> `qc.ccx(1,2,0)` | Applies the CNOTs and a Toffoli (CCX). 
#  | This is the **Recovery Circuit** (also called the Decoder). The key gate is the **Toffoli (CCX)** gate, which acts as a 
#                 "majority vote" or correction mechanism. |
# | `qc.measure(0, 0)` | Measures qubit 0. | Measures the corrected state of the **original data qubit (qubit 0)** 
#                 and stores the result in the classical bit 0. |
# | `simulator = AerSimulator()`<br>...<br>`counts = result.get_counts()` | Runs the circuit. | Executes the circuit 
#       1024 times on the Qiskit **Aer simulator** (a perfect quantum computer simulator, but the error is manually introduced). |
# | `print(counts)` | Output. | Prints the final measurement result counts. |

# ### 3\. The Recovery Mechanism (The "Magic")

# The core of the QEC is the final three gates: `qc.cx(0,1)`, `qc.cx(0,2)`, and `qc.ccx(1,2,0)`.

#   * **Syndrome Measurement (Implicit):** In a full QEC code, you would use CNOTs and ancilla qubits to determine 
#       *which* qubit was flipped (the **syndrome**).
#   * **The Simplified Decoder:** This code uses the following logic:
#     1.  The first two CNOTs: `qc.cx(0,1)` and `qc.cx(0,2)` "undo" the initial encoding.
#     2.  The **Toffoli gate `ccx(1,2,0)`** performs the correction:
#           * It acts on qubit **0** (the data qubit) only if **both** qubits 1 and 2 are $|1\rangle$.
#           * **If a single error occurred:** One of the three qubits will be different from the other two 
#       (e.g., $|101\rangle$ or $|011\rangle$). The Toffoli's controls (1 and 2) will *not* both be $|1\rangle$, so **no correction** 
#       is applied to qubit 0.
#           * **If no error occurred:** The state is $|111\rangle$. The controls (1 and 2) are both $|1\rangle$, so qubit 0 is 
#               flipped from $|1\rangle$ to $|0\rangle$. **Wait, this is an oversimplification\!**

# #### A More Accurate View of the Toffoli in this Context:

# The three gates (`qc.cx(0,1)`, `qc.cx(0,2)`, `qc.ccx(1,2,0)`) together implement a **corrective majority vote on qubit 0**. 
#       If an odd number of qubits were $|1\rangle$, the output on qubit 0 will be $|1\rangle$.

# Since the initial state was $|111\rangle$:

#   * **No Error:** State is $|111\rangle$. Final measurement is **1**.
#   * **One Error (e.g., Qubit 1 flips):** State is $|101\rangle$. After the recovery circuit, the state of Qubit 0 is returned to **1**.
#   * **Two Errors:** The code **fails** to correct two errors.

# ### 4\. Anticipated Results and Exam Questions

# #### Anticipated Output

# The code output shows:

# ```
# Noise introduced on qubit 1
# Measurement results: {'1': 1024}
# ```

# Since the logical state encoded was $|1_L\rangle = |111\rangle$, the expected measurement on the corrected data qubit (qubit 0) 
# is **1**. The fact that the count for '1' is 1024 proves the single bit-flip error on qubit 1 was successfully corrected.

# #### Exam Questions & Suggested Answers

# | Question | Suggested Answer (Be concise and confident) |
# | :--- | :--- |
# | **What is the purpose of this practical?** | To demonstrate a basic **Quantum Error Correction (QEC)** scheme, specifically the 
# **three-qubit Bit-Flip Code**, to protect a quantum state from a single bit-flip error. |
# | **What is the logical state you are protecting?** | We are protecting the state $|1\rangle$. 
# It's encoded into the logical state $|1_L\rangle = |111\rangle$. |
# | **What gates are used for encoding?** | Two **CNOT** gates are used: `CX(0, 1)` and `CX(0, 2)`. 
#     They prepare the entangled state $|111\rangle$ from the initial data qubit $|1\rangle$. |
# | **How do you simulate noise/error?** | Noise is simulated by applying a single **$X$ (NOT) gate** 
#     to a randomly chosen qubit (`error_qubit`). This simulates a **bit-flip error**. |
# | **What gate is central to the correction?** | The **Toffoli gate (CCX)**, along with CNOTs, is used. 
#     It effectively performs a "majority vote" to correct the state of the data qubit (qubit 0). |
# | **Why did the measurement result in '1'?** | The initial state was $|1\rangle$, and despite the single bit-flip error, 
# the **error correction circuit successfully recovered** the original state, which we then measured back as **1**. |
# | **Can this code correct two bit-flip errors?** | **No**. This code is a **minimum redundancy code** and can only correct **one** error. 
# Two errors (e.g., $|001\rangle$) would be interpreted as the state $|0_L\rangle$ with one error, leading to a failure in correction. |
